## Reusable Plates
First, we show how to create a reusable plate and fill it with materials.

In [1]:
import murraylab_tools.echo as mt_echo
import os.path
import numpy as np

# Relevant input and output files. Check these out for examples of input file format.
dilution_inputs  = os.path.join("reusable_plate_examples", "inputs")
dilution_outputs = os.path.join("reusable_plate_examples", "outputs")
plate_file  = os.path.join(dilution_outputs, "reusable_plate.dat") # Keeps track of wells used
output_name = os.path.join(dilution_outputs, "reusable_plate_example") # Output

#Create a Reusable Plate by setting reuse_wells=True
#NOTE: This can also be done by hand by creating a CSV with the columns: well,name,concentration,volume,date

reusable_plate = mt_echo.SourcePlate(filename = plate_file, reuse_wells = True)

#We will add three arbitary materials to the plate CSV
#Create three materials
concentrations = [200, 500]
mat1 = mt_echo.EchoSourceMaterial('Mat1', concentrations[0], 0, reusable_plate)
mat2 = mt_echo.EchoSourceMaterial('Mat2', concentrations[1], 0, reusable_plate)
water= mt_echo.EchoSourceMaterial('water', 1, 0, reusable_plate)

print("reusable_plate.materials_to_add", reusable_plate.materials_to_add)
try:
    reusable_plate.add_material_to_well("A1", mat1, mt_echo.max_volume)
    reusable_plate.add_material_to_well("A2", mat2, mt_echo.max_volume)
except ValueError:
    print("Suppressing an Error due to overfull plates and filling to the maximum allowed instead.")
    reusable_plate.add_material_to_well("A1", mat1, mt_echo.max_volume-reusable_plate.wells_used["A1"][2])
    reusable_plate.add_material_to_well("A2", mat2, mt_echo.max_volume-reusable_plate.wells_used["A2"][2])

print("reusable_plate.materials_to_add", reusable_plate.materials_to_add)
reusable_plate.write_to_file() #Save the file
print("reusable_plate.materials_to_add", reusable_plate.materials_to_add)

print("Plate Volume Variables (nL):")
print("Plate max fill volume:", mt_echo.max_volume)
print("Plate dead volume:", mt_echo.dead_volume)
print("Max Usable volume of each material", mt_echo.max_volume-mt_echo.dead_volume)

reusable_plate.materials_to_add {}
reusable_plate.materials_to_add {('Mat1', 200.0): [('A1', 65000)], ('Mat2', 500.0): [('A2', 65000)]}
reusable_plate.materials_to_add {('Mat1', 200.0): [('A1', 65000)], ('Mat2', 500.0): [('A2', 65000)]}
Plate Volume Variables (nL):
Plate max fill volume: 65000
Plate dead volume: 21000
Max Usable volume of each material 44000


### Once a plate is created, it can be used in multiple experiments as demonstrated below.

This process will generate N_experiments picklists, all using a single source plate. In the initial few experiments, the initially filled wells will be drained. Instructions will then appear asking the user to fill additional wells. After running many times, the source plate will run out of wells and an error will be thrown.

In [2]:
echo_calc = mt_echo.EchoRun(plate = reusable_plate)

N_experiments = 3
for n in range(N_experiments):
    
    #Generate a random experiment in three wells
    random_concentrations = np.random.rand(2)*100
    
    well1 = "A"+str(n+1)
    well2 = "B"+str(n+1)
    echo_calc.add_material_to_well(mat1, well1, random_concentrations[0])
    echo_calc.add_material_to_well(mat2, well1, random_concentrations[1])
    echo_calc.fill_well_with(water, well1) #Fill the rest of well1 with water

    echo_calc.add_material_to_well(mat1, well2, random_concentrations[1])
    echo_calc.add_material_to_well(mat2, well2, random_concentrations[0])
    echo_calc.fill_well_with(water, well2) #Fill the rest of well2 with water

    # Write results
    echo_calc.write_picklist(output_name+str(n))
    
    print("Source Plate Volumes:")
    for well in reusable_plate.wells_used:
        name, conc, vol, date = reusable_plate.wells_used[well]
        print("\tWell:", well, "[", name,"]=", conc, "volume = ", vol, "filled on", date)

Source Plate Volumes:
	Well: A1 [ Mat1 ]= 200.0 volume =  59875 filled on 10/08/2020
	Well: A2 [ Mat2 ]= 500.0 volume =  62950 filled on 10/08/2020
	Well: A03 [ water ]= 1.0 volume =  21000 filled on 10/08/2020
Source Plate Volumes:
	Well: A1 [ Mat1 ]= 200.0 volume =  44700 filled on 10/08/2020
	Well: A2 [ Mat2 ]= 500.0 volume =  56875 filled on 10/08/2020
	Well: A03 [ water ]= 1.0 volume =  21000 filled on 10/08/2020
	Well: A05 [ water ]= 1.0 volume =  21000 filled on 10/08/2020
Source Plate Volumes:
	Well: A1 [ Mat1 ]= 200.0 volume =  21000 filled on 10/08/2020
	Well: A2 [ Mat2 ]= 500.0 volume =  44625 filled on 10/08/2020
	Well: A03 [ water ]= 1.0 volume =  21000 filled on 10/08/2020
	Well: A05 [ water ]= 1.0 volume =  21000 filled on 10/08/2020
	Well: A07 [ Mat1 ]= 200.0 volume =  21000 filled on 10/08/2020
	Well: A09 [ water ]= 1.0 volume =  21000 filled on 10/08/2020
	Well: A10 [ water ]= 1.0 volume =  21000 filled on 10/08/2020


## Refilling Wells

Sometimes, especially with valuable reagents, it makes sense to refill wells in a source plate. This can be done provided that the same material with the same concentration is put into an old well. Below, we refill all the wells A1, A2, and A3 filled to their maximum amounts. If one attempts to overfill a well or add a different material (or a different concentration of the same material) to a well which has already been filled, an error is thrown.

In [3]:
materials = [mat1, mat2]
wells = ["A1", "A2"]
print("Initial Well Volumes:")
for well in ["A1", "A2"]:
    name, conc, vol, date = reusable_plate.wells_used[well]
    print("\tWell:", well, "[", name,"]=", conc, "volume = ", vol, "filled on", date)
    
    
print("\nRefilling...")
#Refill wells
for i in range(len(wells)):
    well = wells[i]
    mat = materials[i]
    name, conc, vol, date = reusable_plate.wells_used[well]
    reusable_plate.add_material_to_well(well, mat, mt_echo.max_volume - vol)

print("\nFinal Volumes after being refilled:")
for well in ["A1", "A2"]:
    name, conc, vol, date = reusable_plate.wells_used[well]
    print("\tWell:", well, "[", name,"]=", conc, "volume = ", vol, "filled on", date)

reusable_plate.write_to_file()

Initial Well Volumes:
	Well: A1 [ Mat1 ]= 200.0 volume =  21000 filled on 10/08/2020
	Well: A2 [ Mat2 ]= 500.0 volume =  44625 filled on 10/08/2020

Refilling...
Refilling Well A1 with additional material Mat1.
Refilling Well A2 with additional material Mat2.

Final Volumes after being refilled:
	Well: A1 [ Mat1 ]= 200.0 volume =  65000 filled on 10/08/2020
	Well: A2 [ Mat2 ]= 500.0 volume =  65000 filled on 10/08/2020
